# How many people are in Heathrow terminal 5 now?
I'm not aware of any footfall counting source, but I think the best way to determine this will be understanding the capacity of flights leaving and arriving, relative to the time now.

### Logic:
The number of people in Heathrow T5 right now is equal to: 
1. The capacity of flights leaving now and for the next 90 minutes __plus__ 
2. the capacity of flights which have landed in the last half an hour, as people don't tend to wait in the airport as long after they have landed.

### Steps:
1. Calculate number of seats on flights leaving t5 at __the present time + 90 minutes__
2. Calculate number of seats on flights which have __landed in the last 30 minutes__ as they don't tend to wait in the airport once they have arrived
3. Understanding the capacity of the planes from the plane type. Will need to match the flight # and plane type the flight number corresponds to

### Assumptions:
1. multiple plane types aren't assigned to the same plane number. i.e. BAW952 will always be an A380


### Blockers:
__Head count in each plane__  
presently I haven't found a source of information that tells you the capacity of a flight. This historical data could be bought from the airlines themselves and predicted
this script uses local UTC time within its filters on an updating datasource so it can be run at any time of day to get the most accurate answer

maybe i could go to an aggregator to source flight fullness. 

seats are in a different airport but the seat capacity seems wrong. 

__Terminal 5 issue__  
gates A1-A23, B32 - B48,  c52 - C66


__change of datasource__
- datasource is scraped from the live JSON that feeds the tables on https://www.heathrow.com/departures and https://www.heathrow.com/arrivals
- since I last looked at this, Heathrow have banned get requests to their server that feeds the website. Very annoying. 
- I've found something at https://flightaware.com/ that looks like it can deliver the same deal. 

In [1]:
import numpy as np
import requests
import json
import datetime
import time
import pandas as pd
from pandas.io.json import json_normalize
from datetime import date
from datetime import timedelta
from bs4 import BeautifulSoup

In [2]:
username = "elliotamcbride"
apiKey = apiKey
fxmlUrl = "https://flightxml.flightaware.com/json/FlightXML2/"
endpoint = 'AirlineFlightSchedules'

#documenation needs the following variables for the payload
#explaination needed for epoch days

today = date.today()
str_today = str(today)
pattern = '%Y-%m-%d'
epoch_today = int(time.mktime(time.strptime(str_today, pattern)))

tomorrow = datetime.date.today() + datetime.timedelta(days = 1)
str_tomorrow = str(tomorrow)
pattern = '%Y-%m-%d'
epoch_tomorrow = int(time.mktime(time.strptime(str_tomorrow, pattern)))

#I have 2 airlines I am interested in as they leave from t5
# I also need anything arriving to heathrow or leaving from heathrow as the payload item won't allow 2 fields

LeavingOrArriving = ['origin', 'destination']
airlines = ['BAW', 'IBE']
x = 0 # to log 

for LoA in LeavingOrArriving:
    for i in airlines:
        payload = {'startDate': 1603917334, 'endDate': 1603929334, LoA: 'EGLL', 'airline': i, 'howMany': 6}
        response = requests.get(fxmlUrl + endpoint, params=payload, auth=(username, apiKey))
    if response.status_code == 200:
        schedule_payload_df[x] = response.json()
        x += 1
        print("Request is good and " + str(LoA) + " is stored in object " + str(x) + " of schedule_payload_df")
    else:
        print("Error executing request")


NameError: name 'schedule_payload_df' is not defined

### turn into one dataframe

In [3]:
DictionaryOfDataFrames = {} 

for i in range(2):
    DictionaryOfDataFrames["group" + str(i)] = json_normalize(schedule_payload_df[i]['AirlineFlightSchedulesResult']['data'])

flight_schedule_dataframe = pd.concat(DictionaryOfDataFrames.values(), ignore_index=True)

flight_schedule_dataframe

NameError: name 'schedule_payload_df' is not defined

In [239]:
flight_schedule_dataframe #cool - now I can wrangle once rather than twice.
    
#flight_schedule_dataframe['DepartureOrArrival'] == np.where(flight_schedule_dataframe.origin == 'EGLL', 'Departure', 'Arrival')

DepartureOrArrival = flight_schedule_dataframe.origin.apply(lambda x : 'Origin' if x == 'EGLL' else 'Departure')

flight_schedule_dataframe['DepartureOrArrival'] = DepartureOrArrival

flight_schedule_dataframe

,ident,actual_ident,departuretime,arrivaltime,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach,DepartureOrArrival
0,IBE7345,BAW105,1603917600,1603943100,EGLL,OMDB,B789,First: No meal / Business: No meal / Economy: ...,8,42,166,Origin
1,IBE7319,BAW57,1603919400,1603959300,EGLL,FAOR,,First: No meal / Business: No meal / Economy: ...,12,48,159,Origin
2,IBE7421,BAW247,1603919700,1603962300,EGLL,SBGR,,First: No meal / Business: No meal / Economy: ...,14,56,227,Origin
3,IBE7321,BAW59,1603920600,1603962900,EGLL,FACT,,First: No meal / Business: No meal / Economy: ...,14,56,227,Origin
4,IBE7370,BAW157,1603922100,1603944300,EGLL,OKBK,,First: No meal / Business: No meal / Economy: ...,8,49,178,Origin
5,IBE7389,BAW194,1603922400,1603954500,KIAH,EGLL,,First: No meal / Business: No meal / Economy: ...,8,49,178,Departure
6,IBE4188,AAL46,1603926300,1603954200,KORD,EGLL,B77W,First: No meal / Business: No meal / Economy: ...,8,52,244,Departure
7,IBE7450,BAW294,1603926600,1603954200,KORD,EGLL,B789,First: No meal / Business: No meal / Economy: ...,8,42,166,Departure
8,IBE7396,BAW206,1603923300,1603953300,KMIA,EGLL,A388,First: No meal / Business: No meal / Economy: ...,10,76,439,Departure
9,IBE4192,AAL50,1603928100,1603960800,KDFW,EGLL,B77W,First: No meal / Business: No meal / Economy: ...,8,52,244,Departure


### notes so far: 
1. seats in these planes are inconsistant some are missing and there is no documentation on whether they are occupied or not so I logged a support ticket. Will readdress - maybe stat significance with average somewhere?
2. Aircraft type is also missing in some instances...All aircrafts have an aircraft type. I may be able to join the actul_ident field on another dataset. 

In [ ]:
flight_schedule_dataframe['departuretime'] = pd.to_datetime(flight_schedule_dataframe['departuretime'],unit='s')

print(flight_schedule_dataframe.head(2))

#### filter flights with valid times

In [242]:
today = date.today()

timenow = datetime.datetime.now()

stringoftimernow = timenow.strftime("%H:%M:%S")

print (str(today))
print(timenow)
print(stringoftimernow)

2020-10-28
2020-10-28 23:56:29.460031
23:56:29


In [244]:
#filter
#create time filter values

timeplus90 = datetime.datetime.now() + timedelta(hours = 1.5)

strtimePlus90 = timeplus90.strftime("%H:%M:%S")

#filter out non-relevant times
shortdf = df[(df.departureTime >= timenow) & (df.departureTime <= strtimePlus90)]

depflights = shortdf.shape[0]

print(depflights)

NameError: name 'df' is not defined